# ADS API - Copernicus Atmosphere Monitoring Service (CAMS)

Reference material:<br> 
GitHub Repository containing examples from ECMWF: https://github.com/ecmwf-projects/copernicus-training

Example: https://github.com/ecmwf-projects/copernicus-training/blob/master/2021-02-Copernicus-ECMWF-data-tutorial.ipynb

Link to Copernicus Services:
- Link to ADS Atmosphere Copernicus: https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview
- Link to CDS Climate Copernicus: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview

Change key/value provided in the .cdsapirc file as explained in the following links:
- Climate: ERA5 https://cds.climate.copernicus.eu/api-how-to
- Atmosphere: CAMS https://ads.atmosphere.copernicus.eu/api-how-to 

Or use the Key/Value in the API request:
<br>
For ADS Data:
- url: https://ads.atmosphere.copernicus.eu/api/v2


For CDS Data:<br>
- url: https://cds.climate.copernicus.eu/api/v2


### Import libraries

In [1]:
import cdsapi
import os
import geopandas as gpd
import xarray as xr
import warnings
warnings.filterwarnings('ignore')
import json
import ipywidgets as widgets

In [2]:
from functions import my_methods

In [3]:
cwd = os.getcwd()

### Set and end date:

Modify **date.json** file to change the date. This file is the same for all the notebooks:

In [4]:
d = open('date.json')
date = json.load(d)
year = date['year']
mais_week = date['mais_week']
rice_week = date['rice_week']
cereal_week = date['cereal_week']

The calendar **manuring_periods** functions allows to create the calendar with corresponding manuring weeks:

In [5]:
calendar = my_methods.manuring_periods(year, mais_week, rice_week, cereal_week)
print(calendar)

{1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: ['2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01'], 10: [], 11: [], 12: [], 13: ['2020-03-23', '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29'], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: ['2020-08-24', '2020-08-25', '2020-08-26', '2020-08-27', '2020-08-28', '2020-08-29', '2020-08-30'], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: [], 52: [], 53: []}


In [6]:
# With this widget is possible to select from the dropdown list the required week
select_week = widgets.Dropdown(
    options=['mais_week', 'rice_week', 'cereal_week'],
    description='name:',
    disabled=False)
select_week

Dropdown(description='name:', options=('mais_week', 'rice_week', 'cereal_week'), value='mais_week')

In [7]:
start_date = calendar[date[select_week.value]][0]
end_date = calendar[date[select_week.value]][-1]

### Import Bounding Box:

In [8]:
area_path = cwd + '/bounding_box/bounding_box_buffer20.gpkg'
area = gpd.read_file(area_path).to_crs(4326)
bounds = area.total_bounds

In [9]:
print(bounds)

[ 8.22236763 44.46930839 11.7723925  46.81902056]


Link to ADS API from (https://ads.atmosphere.copernicus.eu/api-how-to). Use the API **id** and **key** to access the **cdsapi** (put in the **keys.json** file):

In [10]:
f = open('keys.json')
keys = json.load(f)

In [11]:
c_atm = cdsapi.Client(keys["ATM_ID"], keys["ATM_KEY"])

- - -

# ADS Atmosphere Copernicus

# [CAMS - PM2.5 - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for PM2.5 over the study area:

In [12]:
pm25_path = cwd+r'\temp\pm25_cams.nc'
pm25 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'particulate_matter_2.5um',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    pm25_path
    )

2022-03-10 17:14:38,157 INFO Welcome to the CDS
2022-03-10 17:14:38,157 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:14:38,202 INFO Request is queued
2022-03-10 17:14:39,256 INFO Request is running
2022-03-10 17:15:10,689 INFO Request is completed
2022-03-10 17:15:10,689 INFO Downloading https://download-0002.copernicus-atmosphere.eu/cache-compute-0002/cache/data9/adaptor.cams_regional_fc.retrieve-1646928906.2525527-30976-12-ea6388b7-0449-42f1-a4a4-1c1456009808.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\pm25_cams.nc (545.5K)
2022-03-10 17:15:11,115 INFO Download rate 1.3M/s 


Read pm25 .nc file:

In [13]:
pm25 = xr.open_dataset(pm25_path)

In [14]:
pm25_mean = pm25.mean(dim='time')

Write pm25_mean .nc file:

In [15]:
pm25_mean.to_netcdf(cwd+r'\temp\pm25_cams.nc')

- - -

# [CAMS - Ammonia - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for ammonia (NH3) over the study area:

In [16]:
nh3_path = cwd+r'\temp\amm_cams.nc'
nh3 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'ammonia',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    nh3_path
    )

2022-03-10 17:15:11,661 INFO Welcome to the CDS
2022-03-10 17:15:11,661 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:15:11,711 INFO Request is queued
2022-03-10 17:15:12,767 INFO Request is running
2022-03-10 17:15:44,709 INFO Request is completed
2022-03-10 17:15:44,709 INFO Downloading https://download-0000.copernicus-atmosphere.eu/cache-compute-0000/cache/data2/adaptor.cams_regional_fc.retrieve-1646928941.9180713-28209-3-602dc341-c9e7-41ad-bc3b-b67e254c12bc.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST\temp\amm_cams.nc (545.5K)
2022-03-10 17:15:45,107 INFO Download rate 1.3M/s 


Read ammonia .nc file:

In [17]:
nh3 = xr.open_dataset(nh3_path)

Calculate average PM2.5 for the selected days:

In [18]:
nh3_mean = nh3.mean(dim='time')

Write ammonia_mean .nc file:

In [19]:
nh3_mean.to_netcdf(cwd+r'\temp\amm_cams.nc', 'w')

- - -

# [CAMS - Sulphur Dioxide - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for sulphur sioxide (SO2) over the study area:

In [20]:
so2_path = cwd + r'/temp/so2_cams.nc'
so2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'sulphur_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    so2_path
    )

2022-03-10 17:15:45,278 INFO Welcome to the CDS
2022-03-10 17:15:45,288 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:15:45,319 INFO Request is queued
2022-03-10 17:15:46,368 INFO Request is running
2022-03-10 17:16:17,831 INFO Request is completed
2022-03-10 17:16:17,831 INFO Downloading https://download-0002.copernicus-atmosphere.eu/cache-compute-0002/cache/data0/adaptor.cams_regional_fc.retrieve-1646928973.361652-31724-11-7165ca7f-a913-410b-acc8-71905803d38e.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/so2_cams.nc (545.5K)
2022-03-10 17:16:18,197 INFO Download rate 1.5M/s 


Read sulphur dioxide .nc file:

In [21]:
so2 = xr.open_dataset(so2_path)

Calculate average sulphure dioxide concentration for the selected days:

In [22]:
so2_mean = so2.mean(dim='time')

Write so2_mean .nc file:

In [23]:
so2_mean.to_netcdf(so2_path)

- - -

# [CAMS - Nitrogen Dioxide - Analysis](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview) 

Download and write .nc file for nitrogen dioxide (NO2) over the study area:

In [24]:
no2_path = cwd + r'/temp/no2_cams.nc'
no2 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_dioxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no2_path
    )

2022-03-10 17:16:18,349 INFO Welcome to the CDS
2022-03-10 17:16:18,349 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:16:18,398 INFO Request is queued
2022-03-10 17:16:19,448 INFO Request is running
2022-03-10 17:16:50,911 INFO Request is completed
2022-03-10 17:16:50,920 INFO Downloading https://download-0003.copernicus-atmosphere.eu/cache-compute-0003/cache/data3/adaptor.cams_regional_fc.retrieve-1646929006.6607392-22104-4-9cd5ea8b-125f-4c14-bedf-b776ecc35670.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/no2_cams.nc (545.5K)
2022-03-10 17:16:51,310 INFO Download rate 1.4M/s 


Read nitrogen dioxide .nc file:

In [25]:
no2 = xr.open_dataset(no2_path)

Calculate average nitrogen dioxide concentration for the selected days:

In [26]:
no2_mean = no2.mean(dim='time')
no2_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    no2_conc   (level, latitude, longitude) float32 2.349 2.052 ... 11.43 10.38

Write no2_mean.nc file:

In [27]:
no2_mean.to_netcdf(no2_path)

- - -

# [Nitrogen Monoxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for nitrogen monoxide (NO) over the study area:

In [28]:
no_path = cwd + r'/temp/no_cams.nc'
no = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'nitrogen_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    no_path
    )

2022-03-10 17:16:51,522 INFO Welcome to the CDS
2022-03-10 17:16:51,522 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:16:51,562 INFO Request is queued
2022-03-10 17:16:52,613 INFO Request is running
2022-03-10 17:17:24,018 INFO Request is completed
2022-03-10 17:17:24,018 INFO Downloading https://download-0002.copernicus-atmosphere.eu/cache-compute-0002/cache/data9/adaptor.cams_regional_fc.retrieve-1646929040.0711575-31126-13-9e945e61-646d-45de-b3f0-4dc086eb0e44.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/no_cams.nc (545.5K)
2022-03-10 17:17:24,367 INFO Download rate 1.5M/s 


Read nitrogen monoxide .nc file:

In [29]:
no = xr.open_dataset(no_path)

Calculate average nitrogen monoxide concentration for the selected days:

In [30]:
no_mean = no.mean(dim='time')
no_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    no_conc    (level, latitude, longitude) float32 0.1531 0.1132 ... 0.6593

Write no_mean.nc file:

In [31]:
no_mean.to_netcdf(no_path)

- - -

# [Carbon Monoxide](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for carbon monoxide (CO) over the study area:

In [32]:
co_path = cwd + r'/temp/co_cams.nc'
co = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'carbon_monoxide',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    co_path
    )

2022-03-10 17:17:24,538 INFO Welcome to the CDS
2022-03-10 17:17:24,538 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:17:24,638 INFO Request is queued
2022-03-10 17:17:25,690 INFO Request is running
2022-03-10 17:17:57,111 INFO Request is completed
2022-03-10 17:17:57,111 INFO Downloading https://download-0001.copernicus-atmosphere.eu/cache-compute-0001/cache/data9/adaptor.cams_regional_fc.retrieve-1646929072.837717-3344-5-66cfa3cf-32c5-4d2f-ac9e-d3fea122196a.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/co_cams.nc (545.5K)
2022-03-10 17:17:57,511 INFO Download rate 1.3M/s 


Read carbon monoxide .nc file:

In [33]:
co = xr.open_dataset(co_path)

Calculate average carbon monoxide concentration for the selected days:

In [34]:
co_mean = co.mean(dim='time')
co_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    co_conc    (level, latitude, longitude) float32 127.3 125.6 ... 221.8 220.8

Write co_mean.nc file:

In [35]:
co_mean.to_netcdf(co_path)

---

# [Ozone](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for ozone (O3) over the study area:

In [36]:
ozone_path = cwd + r'/temp/o3_cams.nc'
ozone = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'ozone',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    ozone_path
    )

2022-03-10 17:17:57,711 INFO Welcome to the CDS
2022-03-10 17:17:57,711 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:17:57,758 INFO Request is queued
2022-03-10 17:17:58,797 INFO Request is running
2022-03-10 17:18:30,263 INFO Request is completed
2022-03-10 17:18:30,263 INFO Downloading https://download-0002.copernicus-atmosphere.eu/cache-compute-0002/cache/data4/adaptor.cams_regional_fc.retrieve-1646929105.6699758-555-5-2e5e9eef-e276-47e0-828a-26112ed03599.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/o3_cams.nc (545.5K)
2022-03-10 17:18:30,598 INFO Download rate 1.6M/s 


Read carbon monoxide .nc file:

In [37]:
ozone = xr.open_dataset(ozone_path)

Calculate average carbon monoxide concentration for the selected days:

In [38]:
ozone_mean = ozone.mean(dim='time')
ozone_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    o3_conc    (level, latitude, longitude) float32 72.33 73.25 ... 51.65 52.19

Write co_mean.nc file:

In [39]:
ozone_mean.to_netcdf(ozone_path)

---

# [Dust](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for dust over the study area:

In [40]:
dust_path = cwd + r'/temp/dust_cams.nc'
dust = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'variable': 'dust',
        'model': 'ensemble',
        'level': '0',
        'date': start_date+'/'+end_date,
        'type': 'analysis',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'leadtime_hour': '0',
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2],
        ],
        'format': 'netcdf',
    },
    dust_path
    )

2022-03-10 17:18:30,786 INFO Welcome to the CDS
2022-03-10 17:18:30,786 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:18:30,829 INFO Request is queued
2022-03-10 17:18:31,873 INFO Request is running
2022-03-10 17:19:03,306 INFO Request is completed
2022-03-10 17:19:03,306 INFO Downloading https://download-0000.copernicus-atmosphere.eu/cache-compute-0000/cache/data4/adaptor.cams_regional_fc.retrieve-1646929138.8579235-23482-11-ecf5569e-7089-4e14-b361-8909ed613c50.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/dust_cams.nc (545.5K)
2022-03-10 17:19:03,939 INFO Download rate 861.1K/s


Read dust .nc file:

In [41]:
dust = xr.open_dataset(dust_path)

Calculate average dust concentration for the selected days:

In [42]:
dust_mean = dust.mean(dim='time')
dust_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    dust       (level, latitude, longitude) float32 0.365 0.3459 ... 0.1807

Write dust.nc file:

In [43]:
dust_mean.to_netcdf(dust_path)

# [PM10](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for PM10 over the study area:

In [44]:
pm10_path = cwd + r'/temp/pm10_cams.nc'
pm10 = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'particulate_matter_10um',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    pm10_path)

2022-03-10 17:19:04,164 INFO Welcome to the CDS
2022-03-10 17:19:04,164 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:19:04,230 INFO Request is queued
2022-03-10 17:19:05,273 INFO Request is running
2022-03-10 17:19:36,710 INFO Request is completed
2022-03-10 17:19:36,710 INFO Downloading https://download-0000.copernicus-atmosphere.eu/cache-compute-0000/cache/data9/adaptor.cams_regional_fc.retrieve-1646929171.9935641-28844-5-87fad345-5067-4371-96f5-e3501a5b32b1.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/pm10_cams.nc (545.5K)
2022-03-10 17:19:36,940 INFO Download rate 2.3M/s 


Read PM10 .nc file:

In [45]:
pm10 = xr.open_dataset(pm10_path)

Calculate average PM10 concentration for the selected days:

In [46]:
pm10_mean = pm10.mean(dim='time')
pm10_mean

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude  (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude   (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level      (level) float32 0.0
Data variables:
    pm10_conc  (level, latitude, longitude) float32 4.338 4.033 ... 15.35 15.27

Write pm10_mean.nc file:

In [47]:
pm10_mean.to_netcdf(pm10_path)

# [Non Methane VOCs (NMVOCs)](https://ads.atmosphere.copernicus.eu/cdsapp#!/dataset/cams-europe-air-quality-forecasts?tab=overview)

Download and write .nc file for non-methane volatile organic compoungs(NMVOCs) over the study area:

In [48]:
nmvocs_path = cwd + r'/temp/nmvocs_cams.nc'
nmvocs = c_atm.retrieve(
    'cams-europe-air-quality-forecasts',
    {
        'model': 'ensemble',
        'date': start_date+'/'+end_date,
        'format': 'netcdf',
        'level': '0',
        'type': 'analysis',
        'variable': 'non_methane_vocs',
        'leadtime_hour': '0',
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            bounds[3], bounds[0], bounds[1],
            bounds[2]],
        'format': 'netcdf'
    },
    nmvocs_path)

2022-03-10 17:19:37,122 INFO Welcome to the CDS
2022-03-10 17:19:37,122 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-europe-air-quality-forecasts
2022-03-10 17:19:37,173 INFO Request is queued
2022-03-10 17:19:38,215 INFO Request is running
2022-03-10 17:20:09,726 INFO Request is completed
2022-03-10 17:20:09,726 INFO Downloading https://download-0000.copernicus-atmosphere.eu/cache-compute-0000/cache/data4/adaptor.cams_regional_fc.retrieve-1646929205.1691244-23280-11-8164ee23-e734-4d5d-9ac6-48e7fc3c0384.nc to C:\Users\Administrator\OneDrive - Politecnico di Milano\WP2\D-DUST/temp/nmvocs_cams.nc (545.5K)
2022-03-10 17:20:10,102 INFO Download rate 1.5M/s 


Read nmvocs .nc file:

In [49]:
nmvocs = xr.open_dataset(nmvocs_path)

Calculate average NMVOCs concentration for the selected days:

In [50]:
nmvocs_mean = nmvocs.mean(dim='time')
nmvocs_mean

<xarray.Dataset>
Dimensions:     (longitude: 36, latitude: 23, level: 1)
Coordinates:
  * longitude   (longitude) float32 8.25 8.35 8.45 8.55 ... 11.55 11.65 11.75
  * latitude    (latitude) float32 46.75 46.65 46.55 46.45 ... 44.75 44.65 44.55
  * level       (level) float32 0.0
Data variables:
    nmvoc_conc  (level, latitude, longitude) float32 3.014 2.781 ... 13.59 12.8

Write nmvocs_mean .nc file:

In [51]:
nmvocs_mean.to_netcdf(nmvocs_path)